In [176]:
import collections
import itertools

In [177]:
sos = '<s>'
eos = '</s>'
def read_docs(dataset_year: str, max_docs_count: int):
  is_doc = False
  docs_count = 0

  with open(f'dataset/cg.Folha.{dataset_year}', encoding='windows-1252') as file: 
    for line in file:
      if docs_count >= max_docs_count:
        break

      if line.startswith('<DOC>'):
        is_doc = True
        continue

      if line.startswith('</DOC>'):
        is_doc = False
        docs_count += 1
        continue

      if is_doc is False:
        continue

      if line.startswith(sos):
        yield (sos, sos)
      elif line.startswith(eos):
        yield (eos, eos)
      else:
        splitted_line = line.split('\t')
        if len(splitted_line) == 2:
          word = splitted_line[0]
          tag = splitted_line[1].split()[1]
          yield (word, tag)

In [178]:
def get_transition_counter(tags):
  current_it, next_it = itertools.tee(tags)
  next(next_it)

  counter = collections.Counter(list(zip(current_it, next_it)))

  return counter

In [179]:
def train(dataset):
  word_tag_counter = collections.Counter() # TODO: Counter 'B'
  tag_counter = collections.Counter() # TODO: Counter 'B'
  transition_counter = collections.Counter() # TODO: Counter 'A'
  sentence_tags = []

  for word, tag in dataset:
    tag_counter.update({ tag: 1 })
    word_tag_counter.update({ (word, tag): 1 })
    
    sentence_tags.append(tag)
    if tag == eos:
      transition_counter += get_transition_counter(sentence_tags)
      sentence_tags = []

  return {
    'word_tag_counter': word_tag_counter,
    'tag_counter': tag_counter,
    'transition_counter': transition_counter
  }

In [180]:
def emission_probability(model, word_tag):
  return model['word_tag_counter'][word_tag] / model['tag_counter'][word_tag[1]]

In [181]:
def transition_probability(model, tag_transition):
  return model['transition_counter'][tag_transition] / model['tag_counter'][tag_transition[0]]

In [182]:
def initial_probability(model):
  initial_prob = { sos: 1 }
  for tag in model['tag_counter'].keys():
    if tag == sos:
      continue
    
    initial_prob.update({ tag: 0 })

  return initial_prob